In [169]:
%%capture
%run 02_account.ipynb

In [170]:
from objsize import get_deep_size as get_size

## Block

In [238]:
class Block: 
    def __init__(self, txs, prev_hash, number, time):
        self.txs       = self.val_txs(txs)
        self.prev_hash = prev_hash
        self.number    = number
        self.time      = time
        self.mined     = False
            
    def add(self, tx):
        assert          val_tx(tx), 'tx validation failed'
        assert not self.is_dup(tx), 'tx is duplicated'
        return tx
            
    def is_dup(self, tx):
        tx_hashes = [tx.hash for tx in self.txs]
        return tx.hash in tx_hashes
    
    def val_txs(self, txs):
        if False not in [val_tx(tx) for tx in txs]: return txs
        else: raise Exception('TXs validation failed')
        
    def __str__(self): 
        block_str = ('time:\t\t'        + time.ctime(self.time)+
                     '\nnumber:\t\t'    + str(self.number)+
                     '\nprev_hash:\t'   + ph(self.prev_hash)+
                     '\n\ntxs:\t\t\n\n' + txs2str(self.txs))
        if self.mined: 
            block_str += ('\ndiff:\t\t'  +str(self.diff)+
                          '\nreward:\t\t'+str(self.reward)+
                          '\nnonce:\t\t' +str(self.nonce)) 
        return block_str
    def __bytes__(self): return (txs2str(self.txs)+
                                 self.prev_hash+
                                 str(self.number)+
                                 str(self.time)).encode()

In [239]:
acc1,acc2 = Account(),Account()

In [240]:
val_tx(tx1)

True

In [241]:
tx1 = acc1.signed_tx(acc2, 24)
tx2 = acc2.signed_tx(acc1, 18)

block = Block([tx1,tx2], rh(), 2, time.time()); print(block)

time:		Fri Mar 26 20:12:55 2021
number:		2
prev_hash:	💪 442daa2782a836be...c2d

txs:		

time:	Fri Mar 26 20:12:55 2021
from:	🔕 0xaF22DC9de8D225...4BD
to:	📥 0x7Fe8F6cf70Ea90...FB2
value:	24 ether
nonce:	0
hash:	📖 706d5adeab17556b...fd8
signed:	true

time:	Fri Mar 26 20:12:55 2021
from:	📥 0x7Fe8F6cf70Ea90...FB2
to:	🔕 0xaF22DC9de8D225...4BD
value:	18 ether
nonce:	0
hash:	📼 96a4424c063f7607...c8d
signed:	true



If anything in the tx is changed, like increasing the value to send, the signature should become invalid and the block creation throws an exception.

In [242]:
tx2.value = 1800

val_failed = False
try:    Block([tx1,tx2], rh(), 2, time.time())
except: val_failed = True
assert  val_failed

## Mining

In [243]:
class Miner:
    def __init__(self, acc): 
        self.acc = acc
        self.pub = acc.pub
        
    def coinbase(self, reward): 
        return self.acc.sign(TX(self.pub, self.pub, reward, self.acc.nonce))
    
    def mine(self, block, diff, reward, attempts=1000):
        mb = deepcopy(block)
        mb.add(self.acc.sign(self.coinbase(reward)))
        mb_b = bytes(mb)
        nonce = 0
        for i in range(attempts):
            candidate   = mb_b + str(nonce).encode()
            candidate_h = sha256(candidate).hexdigest()
            if candidate_h[:diff] == '0'*diff: break
            nonce += 1
        mb.diff   = diff
        mb.reward = reward
        mb.nonce  = nonce
        mb.mined  = True
        return mb
    
miner = Miner(acc1)
mb = miner.mine(block, 2, 100); print(mb)

time:		Fri Mar 26 20:12:55 2021
number:		2
prev_hash:	💪 442daa2782a836be...c2d

txs:		

time:	Fri Mar 26 20:12:55 2021
from:	🔕 0xaF22DC9de8D225...4BD
to:	📥 0x7Fe8F6cf70Ea90...FB2
value:	24 ether
nonce:	0
hash:	📖 706d5adeab17556b...fd8
signed:	true

time:	Fri Mar 26 20:12:55 2021
from:	📥 0x7Fe8F6cf70Ea90...FB2
to:	🔕 0xaF22DC9de8D225...4BD
value:	1800 ether
nonce:	0
hash:	🕊 e43571999aa87b94...cf6
signed:	true

diff:		2
reward:		100
nonce:		171
